In [2]:
### https://adventofcode.com/2022/day/11
# AoC 2022 - 11_1
import re
import numpy as np

class Monkey:
    def __init__(self, name):
        self.monkey_id = name
        self.items = []
        self.operation = ['+', 0]
        self.divisible = 1
        self.next_monkey = [-1, -1]
        self.inspection_count = 0
  
    def get_values(self, text: str, merge=True):
        return int(''.join(re.findall(r'\b\d+\b', text))) if merge else \
            [int(item) for item in re.findall(r'\b\d+\b', text)]
        
    def parse_monkey(self, _line):
        attr, properties = _line.split(': ')
        if attr == 'Starting items':
            self.items = self.get_values(properties, False)
        if attr == 'Operation':
            if properties.endswith('old'):
                self.operation[0] = '**'
                self.operation[1] = 2
            else:
                self.operation[0] = ''.join([op for op in ['*', '/', '+', '-' ] if op in properties])
                self.operation[1] = self.get_values(properties)
        if attr == 'Test':
            self.divisible = self.get_values(properties)
        if attr == 'If true':
            self.next_monkey[0] = self.get_values(properties)
        if attr == 'If false':
            self.next_monkey[1] = self.get_values(properties)


        
def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]


def monkeys_play(_monkeys: dict, _rounds: int):
    for _round in range(1, _rounds + 1):
        init_op = {
            '+': lambda x, y: x + y,
            '-': lambda x, y: x - y,
            '*': lambda x, y: x * y,
            '/': lambda x, y: x / y,
            '**': lambda x, y: x ** y
        }
        for _id, monkey in _monkeys.items():
            items = monkey.items
            if items:
                for item in items:
                    monkey.inspection_count += 1
                    op = monkey.operation
                    item = init_op[op[0]](item, op[1]) // 3
                    idx = 0 if item % monkey.divisible == 0 else 1
                    next_monkey = _monkeys[str(monkey.next_monkey[idx])]
                    next_monkey.items.append(item)
                monkey.items = []

                
input_data = "aoc2022_data/aoc2022_11.txt"
lines = get_input_data(input_data)

num_rounds = 20
number_most_active = 2
monkeys = dict()

for line in lines:
    if line:
        if line.endswith(':'):
            monkey_name = ''.join(re.findall(r'\b\d+\b', line))
            monkey = Monkey(monkey_name)
        else:
            monkey.parse_monkey(line)
            if 'If false: throw to monkey' in line:
                monkeys[str(monkey.monkey_id)] = monkey

monkeys_play(monkeys, num_rounds)

inspections = []

for monkey in monkeys:
    inspections.append(monkeys[monkey].inspection_count)
    print(f"Monkey {monkeys[monkey].monkey_id}: inpected {monkeys[monkey].inspection_count} times.")

monkey_business = np.prod(sorted(inspections, reverse=True)[:number_most_active])

print(f"Level of monkey business after {num_rounds} rounds is for the {number_most_active} " \
      f"most active monkeys: {monkey_business}.")



Monkey 0: inpected 318 times.
Monkey 1: inpected 296 times.
Monkey 2: inpected 35 times.
Monkey 3: inpected 307 times.
Monkey 4: inpected 39 times.
Monkey 5: inpected 337 times.
Monkey 6: inpected 336 times.
Monkey 7: inpected 40 times.
Level of monkey business after 20 rounds is for the 2 most active monkeys: 113232.


In [3]:
# AoC 2022 - 11_2
import re
import numpy as np
import math

class Monkey:
    def __init__(self, name):
        self.monkey_id = name
        self.items = []
        self.operation = ['+', 0]
        self.divisible = 1
        self.next_monkey = [-1, -1]
        self.inspection_count = 0
  
    def get_values(self, text: str, merge=True):
        return int(''.join(re.findall(r'\b\d+\b', text))) if merge else \
            [int(item) for item in re.findall(r'\b\d+\b', text)]
        
    def parse_monkey(self, _line):
        attr, properties = _line.split(': ')
        if attr == 'Starting items':
            self.items = self.get_values(properties, False)
        if attr == 'Operation':
            if properties.endswith('old'):
                self.operation[0] = '**'
                self.operation[1] = 2
            else:
                self.operation[0] = ''.join([op for op in ['*', '/', '+', '-' ] if op in properties])
                self.operation[1] = self.get_values(properties)
        if attr == 'Test':
            self.divisible = self.get_values(properties)
        if attr == 'If true':
            self.next_monkey[0] = self.get_values(properties)
        if attr == 'If false':
            self.next_monkey[1] = self.get_values(properties)


        
def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]


def monkeys_play(_monkeys: dict, _rounds: int):    
    # see https://de.wikipedia.org/wiki/Prime_Restklassengruppe
    prim_factor = np.prod([monkey.divisible for _id, monkey in _monkeys.items()])
    for _round in range(1, _rounds + 1):
        init_op = {
            '+': lambda x, y: x + y,
            '-': lambda x, y: x - y,
            '*': lambda x, y: x * y,
            '/': lambda x, y: x / y,
            '**': lambda x, y: x ** y
        }
        for _id, monkey in _monkeys.items():
            items = monkey.items
            if items:
                for item in items:
                    monkey.inspection_count += 1
                    op = monkey.operation
                    item = init_op[op[0]](item, op[1])
                    idx = 0 if item % monkey.divisible == 0 else 1
                    next_monkey = _monkeys[str(monkey.next_monkey[idx])]
                    next_monkey.items.append(item % prim_factor)
                monkey.items = []

                
input_data = "aoc2022_data/aoc2022_11.txt"
lines = get_input_data(input_data)

num_rounds = 10000
number_most_active = 2
monkeys = dict()

for line in lines:
    if line:
        if line.endswith(':'):
            monkey_name = ''.join(re.findall(r'\b\d+\b', line))
            monkey = Monkey(monkey_name)
        else:
            monkey.parse_monkey(line)
            if 'If false: throw to monkey' in line:
                monkeys[str(monkey.monkey_id)] = monkey

monkeys_play(monkeys, num_rounds)

inspections = []

for monkey in monkeys:
    inspections.append(monkeys[monkey].inspection_count)
    print(f"Monkey {monkeys[monkey].monkey_id}: inpected {monkeys[monkey].inspection_count} times.")

monkey_business = np.prod(sorted(inspections, reverse=True)[:number_most_active])

print(f"Level of monkey business after {num_rounds} rounds is for the {number_most_active} " \
      f"most active monkeys: {monkey_business}.")

Monkey 0: inpected 171832 times.
Monkey 1: inpected 12942 times.
Monkey 2: inpected 167042 times.
Monkey 3: inpected 12599 times.
Monkey 4: inpected 167043 times.
Monkey 5: inpected 172863 times.
Monkey 6: inpected 12961 times.
Monkey 7: inpected 160272 times.
Level of monkey business after 10000 rounds is for the 2 most active monkeys: 29703395016.


In [4]:
### https://adventofcode.com/2022/day/12
# AoC 2022 - 12_1

# TODO later, NOT FINISHED JET

import itertools
import numpy as np


def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]

    
def get_nodes(_matrix):
    return list(itertools.chain.from_iterable(_matrix))
    
    
def create_graph(_matrix):
    start = (0, 0)
    end = (0, 0)
    num_rows = len(_matrix[:, 0])
    num_cols = len(_matrix[:, 0])
    nodes = []
    print(num_rows, num_cols)
    
    for _row in range(num_rows):
        for _col in range(num_cols):
            if _matrix[_row, _col] == 'S':
                start = (_row, _col)
            if _matrix[_row, _col] == 'E':
                end = (_row, _col)
            else:
                curr_node = _matrix[_row][_col]
                neighbor = None
                # up
                if _row > 0:
                    neighbor = _matrix[_row - 1][_col]
                    if abs(ord(neighbor) - ord(node)) == 1:
                        nodes.append(neighbor)
                # down
                if _row < num_rows - 1:
                    neighbor = _matrix[_row + 1][_col]
                    
                # left
                if _col > 0:
                    neighbor = _matrix[_row][_col - 1]
                    
                # down
                if _col < num_cols - 1:
                    neighbor = _matrix[_row][_col + 1]
                    
                
    
    
input_data = "aoc2022_data/aoc2022_12.txt"
input_data = "aoc2022_data/example.txt"
lines = get_input_data(input_data)
node_matrix = np.array([[*line] for line in lines])
nodes = get_nodes(node_matrix)

print(node_matrix)
print(nodes)
create_graph(node_matrix)





[list(['8', ',', '4', ' ', '-', '>', ' ', '8', ',', '6', ' ', '-', '>', ' ', '6', ',', '6'])
 list(['1', '3', ',', '4', ' ', '-', '>', ' ', '1', '2', ',', '4', ' ', '-', '>', ' ', '1', '2', ',', '9', ' ', '-', '>', ' ', '4', ',', '9'])]
['8', ',', '4', ' ', '-', '>', ' ', '8', ',', '6', ' ', '-', '>', ' ', '6', ',', '6', '1', '3', ',', '4', ' ', '-', '>', ' ', '1', '2', ',', '4', ' ', '-', '>', ' ', '1', '2', ',', '9', ' ', '-', '>', ' ', '4', ',', '9']


<ipython-input-4-91c9ce88b4e4>:59: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  node_matrix = np.array([[*line] for line in lines])


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
# AoC 2022 - 12_2

# TODO later

In [ ]:
### https://adventofcode.com/2022/day/13
# AoC 2022 - 13_1

# TODO later

In [ ]:
# AoC 2022 - 13_2

# TODO later

In [ ]:
### https://adventofcode.com/2022/day/14
# AoC 2022 - 14_1

import numpy as np

        
def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]
    
    
def apply_missing_fields(x_list, y_list):
    x_coord, y_coord = [], []
    for i in range(len(x_list) - 1):
        diff_x = x_list[i] - x_list[i + 1]
        diff_x_abs = abs(diff_x)
        
        diff_y = y_list[i] - y_list[i + 1]
        diff_y_abs = abs(diff_y)
        
        x_coord.append(x_list[i])
        y_coord.append(y_list[i])
        
        if diff_x_abs > 1 and diff_y_abs <= 1:
            for j in range(1, diff_x_abs):
                if diff_x > 1:
                    x_coord.append(x_list[i] - j)
                    y_coord.append(y_list[i])                
                if diff_x < -1:
                    x_coord.append(x_list[i] + j)
                    y_coord.append(y_list[i])  
                                   
        if diff_y_abs > 1 and diff_x_abs <= 1:
            for j in range(1, diff_y_abs):
                if diff_y > 1:
                    y_coord.append(y_list[i] - j)
                    x_coord.append(x_list[i])                
                if diff_y < -1:
                    y_coord.append(y_list[i] + j)
                    x_coord.append(x_list[i]) 
    x_coord.append(x_list[-1]) 
    y_coord.append(y_list[-1]) 
    return x_coord, y_coord


def scan_paths(_lines: str, _abyss_space: int):
    max_x, max_y, _paths = 0, 0, []
    for _line in _lines:
        _line = [int(item) for items in _line.split(' -> ') for item in items.split(',')]
        x_path = _line[::2]
        y_path = _line[1::2]
        x_path, y_path = apply_missing_fields(x_path, y_path)
        max_x, max_y = max(max_x, max(x_path)), max(max_y, max(y_path))
        _paths.append([y_path, x_path])        
    arr = np.chararray((max_y + abyss_space, max_x + abyss_space))
    arr[:] = '.'
    return arr, _paths
    

def draw_paths(_scan,_paths: list):
    for _path in _paths:        
        for row, col in zip(_path[0], _path[1]):
            _scan[row][col] = '#'

            
def sand_runs(scan, start_col):
    count = 0    
    num_rows = len(scan[:,start_col])
    row = 0
    col = start_col
    while True:
        count_start = count
        bottom_blocked = scan[row + 1][col].decode() != '.'
        left_blocked = scan[row + 1][col - 1].decode() != '.'
        right_blocked = scan[row + 1] [col + 1].decode() != '.'
        
        if bottom_blocked:                    
            if left_blocked and not right_blocked:
                col += 1
            elif left_blocked and right_blocked:
                scan[row][col] = 'o'
                count += 1
                row, col = 0, start_col
                continue
            elif not left_blocked:
                col -= 1
                
        row += 1
        if row >= num_rows - 1:
            return scan, count           
           

input_data = "aoc2022_data/aoc2022_14.txt"
lines = get_input_data(input_data)
abyss_space = 10
sand_start_col = 500


ground_scan, paths = scan_paths(lines, abyss_space)
draw_paths(ground_scan, paths)
ground_scan, _count = sand_runs(ground_scan, sand_start_col)

# for line in ground_scan.decode():
#     print(''.join(line))

print(f"{_count} units of sand come to rest before sand starts flowing into the abyss below.")


In [16]:
mischgemüsemischgemüse# AoC 2022 - 14_2

import numpy as np

        
def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]
    
    
def apply_missing_fields(x_list, y_list):
    x_coord, y_coord = [], []
    for i in range(len(x_list) - 1):
        diff_x = x_list[i] - x_list[i + 1]
        diff_x_abs = abs(diff_x)
        
        diff_y = y_list[i] - y_list[i + 1]
        diff_y_abs = abs(diff_y)
        
        x_coord.append(x_list[i])
        y_coord.append(y_list[i])
        
        if diff_x_abs > 1 and diff_y_abs <= 1:
            for j in range(1, diff_x_abs):
                if diff_x > 1:
                    x_coord.append(x_list[i] - j)
                    y_coord.append(y_list[i])                
                if diff_x < -1:
                    x_coord.append(x_list[i] + j)
                    y_coord.append(y_list[i])  
                                   
        if diff_y_abs > 1 and diff_x_abs <= 1:
            for j in range(1, diff_y_abs):
                if diff_y > 1:
                    y_coord.append(y_list[i] - j)
                    x_coord.append(x_list[i])                
                if diff_y < -1:
                    y_coord.append(y_list[i] + j)
                    x_coord.append(x_list[i]) 
    x_coord.append(x_list[-1]) 
    y_coord.append(y_list[-1]) 
    return x_coord, y_coord


def scan_paths(_lines: str, space_to_ground: int):
    max_x, max_y, _paths = 0, 0, []
    for _line in _lines:
        _line = [int(item) for items in _line.split(' -> ') for item in items.split(',')]
        x_path = _line[::2]
        y_path = _line[1::2]
        x_path, y_path = apply_missing_fields(x_path, y_path)
        max_x = max(max_x, max(x_path))        
        max_y = max(max_y, max(y_path))
        _paths.append([y_path, x_path])     
    max_y += space_to_ground + 1
    estimated_x_range = max_x + 2 * max_y
    arr = np.chararray((max_y, max_x + estimated_x_range))
    arr[:] = '.'
    ground = len(arr[-1, :])
    arr[-1,:] = ['#'] * ground        
    return arr, _paths, estimated_x_range // 2 + 1
    

def draw_paths(_scan,_paths: list, left_x):
    for _path in _paths:        
        for row, col in zip(_path[0], _path[1]):
            _scan[row][col + left_x] = '#'

            
def sand_runs(scan, start_col):
    count = 0    
    num_rows = len(scan[:,start_col])
    row = 0
    col = start_col
    while True:
        count_start = count
        bottom_blocked = scan[row + 1][col].decode() != '.'
        left_blocked = scan[row + 1][col - 1].decode() != '.'
        right_blocked = scan[row + 1] [col + 1].decode() != '.'
        
        if bottom_blocked:                    
            if left_blocked and not right_blocked:
                col += 1
            elif left_blocked and right_blocked:
                scan[row][col] = 'o'
                count += 1
                if row == 0:
                    return scan, count  
                row, col = 0, start_col
                continue
            elif not left_blocked:
                col -= 1        
        row += 1
        if row >= num_rows - 1:
            row = 0  
           
        

input_data = "aoc2022_data/aoc2022_14.txt"
out_file = "results/aoc2022_14-2_picture.txt"
lines = get_input_data(input_data)
sand_start_col = 500
space_to_ground = 2


ground_scan, paths, left_x_range = scan_paths(lines, space_to_ground)
sand_start_col += left_x_range
draw_paths(ground_scan, paths, left_x_range)
ground_scan, _count = sand_runs(ground_scan, sand_start_col)

with open(out_file, 'w') as f_out:    
    for line in ground_scan.decode():
        f_out.write(''.join(line) + '\n')

print(f"{_count} units of sand come to rest.")


24377 units of sand come to rest.


In [ ]:
### https://adventofcode.com/2022/day/15
# AoC 2022 - 15_1

# TODO later

In [ ]:
# AoC 2022 - 15_2

# TODO later

In [128]:
### https://adventofcode.com/2022/day/16
# AoC 2022 - 16_1

### DRAFT #1

import re
        
def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]
           

def create_graph(_lines):
    _graph = dict()
    parse_out = ";|,|rate|="
    for line in _lines:
        line = line.split()
        node = re.sub(parse_out, "", line[1])
        rate = int(re.sub(parse_out, "", line[4]))
        neighbors = [re.sub(parse_out, "", l) for l in line[9:]]        
        _graph[node] = {'neighbors': neighbors, 'rate': rate}
    return _graph


def search_paths(graph, start, end, steps, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    paths = []
    for node in graph[start]['neighbors']:
        if len(path) < steps:
            newpaths = search_paths(graph, node, end, steps, path)
            for newpath in newpaths:
                    paths.append(newpath)
    return paths 

    
def get_all_node_pairs(_graph):
    combinations = []
    nodes = list(_graph.keys())
    for node in nodes:
        nodes = list(_graph.keys())
        nodes.pop(nodes.index(node))                  
        combinations += list(zip([node]*len(nodes), nodes))
    return list(set(combinations))


def get_pressure(_path, graph, time):
    sum_pressure = 0
    for idx, node in enumerate(_path):
        time -= 1
        if idx > 0:
            prev_node = _path[idx - 1]
            parent_neighbors = graph[prev_node]['neighbors']
            time -= len(parent_neighbors[parent_neighbors.index(node):])
        sum_pressure += time * graph[node]['rate']
    return [sum_pressure]


input_data = "aoc2022_data/aoc2022_16.txt"
input_data = "aoc2022_data/example.txt"
lines = get_input_data(input_data)
time = 30

graph = create_graph(lines)
node_pairs = get_all_node_pairs(graph)
all_paths = []
for start, end in node_pairs:
    all_paths += search_paths(graph, start, end, time)
    
print("PATHS: ", all_paths)
print(['BB', 'CC', 'DD', 'EE', 'HH', 'JJ'] in paths)

pressure = []
for _path in all_path:
    pressure += get_pressure(_path, graph, time)
    
print(pressure)    
print(max(pressure))

KeyboardInterrupt: 

In [23]:
### https://adventofcode.com/2022/day/16
# AoC 2022 - 16_1

### DRAFT #2

import re
from itertools import permutations
        
def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]
           

def create_graph(_lines):
    _graph = dict()
    parse_out = ";|,|rate|="
    for line in _lines:
        line = line.split()
        node = re.sub(parse_out, "", line[1])
        rate = int(re.sub(parse_out, "", line[4]))
        neighbors = [re.sub(parse_out, "", l) for l in line[9:]]        
        _graph[node] = {'neighbors': neighbors, 'rate': rate}
    return _graph


def search_paths(_graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    paths = []
    for node in _graph[start]['neighbors']:
        if node not in path:
            newpaths = search_paths(_graph, node, end, path)
            for newpath in newpaths:
                    paths.append(newpath)
    return paths


def get_shortest_path_costs(_graph, start, end):
    return min([len(_path) for _path in search_paths(_graph, start, end)])
    
    
def get_all_node_pairs(_graph):
    combinations = []
    nodes = list(_graph.keys())
    for node in nodes:
        nodes = list(_graph.keys())
        nodes.pop(nodes.index(node))                  
        combinations += list(zip([node]*len(nodes), nodes))
    return list(set(combinations))


def get_pressure(_path, _graph, time):
    sum_pressure = 0
    for idx, node in enumerate(_path):
        time -= 1
        if idx > 0:
            prev_node = _path[idx - 1]
            parent_neighbors = _graph[prev_node]['neighbors']
            time -= len(parent_neighbors[parent_neighbors.index(node):])
        sum_pressure += time * _graph[node]['rate']
    return [sum_pressure]


def calc_pressure(_graph, node, _time):
    return _time * _graph[node]['rate']


def get_use_lost_items(_list, pairs_use_loss):
    use_lost_items = dict()
    for item in _list:
        use_lost_items[item] = pairs_use_loss[item]
    return use_lost_items    


# use greedy the node pair with maximal use-loss-ratio
def check_upstream_downstream(_graph, pairs_use_loss, pairs_costs, time_remains, _paths, opened, sum_vals):
    for node_pair_id, value in pairs_use_loss.items():     
        start, end = node_pair_id.split('_')
        for node in [start, end]:
            if node not in opened:
                time_remains -= 1
                if node == end:
                    time_remains -= pairs_costs[node_pair_id]
                opened.append(node)
                sum_vals += calc_pressure(_graph, node, time_remains)
        if pairs_use_loss == [] or time_remains <= 0:
            return _paths[opened]   
        _paths['_'.join(opened)] = sum_vals
        
        upstream_list = [item for item in list(pairs_use_loss.keys()) if item.endswith(start) and not item.startswith(end) and not item in opened]
        upstream_use_loss = get_use_lost_items(upstream_list, pairs_use_loss)
        
        downstream_list = [item for item in list(pairs_use_loss.keys()) if item.startswith(end) and not item.endswith(start) and not item in opened]
        downstream_use_loss = get_use_lost_items(downstream_list, pairs_use_loss)
        
#         print(upstream_use_loss)
#         print(downstream_use_loss)
        paths = check_upstream_downstream(_graph, upstream_use_loss, pairs_costs, time_remains, _paths, opened, sum_vals)
        check_upstream_downstream(_graph, downstream_use_loss, pairs_costs, time_remains, _paths, opened, sum_vals)
    return _paths
        
    
# approximation of benefit and costs to select optimal path greedy
def get_optimal_path(_graph, pairs: list, time_remains, node_pairs_costs=dict(), node_pairs_benefit=dict(), node_pairs_use_loss=dict()):
    for start, end in node_pairs:
        node_pairs_costs[f"{start}_{end}"] = get_shortest_path_costs(_graph, start, end)
        node_pairs_benefit[f"{start}_{end}"] = (time_remains - 1) * _graph[start]['rate'] + (time_remains - node_pairs_costs[f"{start}_{end}"] - 1) * _graph[end]['rate']
        node_pairs_use_loss[f"{start}_{end}"] = node_pairs_benefit[f"{start}_{end}"] / node_pairs_costs[f"{start}_{end}"]
    node_pairs_costs = dict(sorted(node_pairs_costs.items(), key=lambda item: item[1]))
    node_pairs_benefit = dict(sorted(node_pairs_benefit.items(), key=lambda item: item[1],reverse=True))
    node_pairs_use_loss = dict(sorted(node_pairs_use_loss.items(), key=lambda item: item[1],reverse=True))
#     print(node_pairs_use_loss)
    _paths = dict()
    return check_upstream_downstream(_graph, node_pairs_use_loss, node_pairs_costs, time_remains, _paths, [], 0)
    
    
input_data = "aoc2022_data/aoc2022_16.txt"
input_data = "aoc2022_data/example.txt"
lines = get_input_data(input_data)
time = 30

graph = create_graph(lines)
node_pairs = get_all_node_pairs(graph)
optimal_paths = get_optimal_path(graph, node_pairs, time)

print(optimal_paths)

print(max(list(optimal_paths.items())))


{'DD_EE': 684, 'DD_EE_CC': 740, 'DD_EE_CC_BB': 1104, 'DD_EE_CC_BB_JJ': 1692, 'DD_EE_CC_BB_JJ_AA': 1692, 'DD_EE_CC_BB_JJ_AA_HH': 2308, 'DD_EE_CC_BB_JJ_AA_HH_FF': 2308, 'DD_EE_CC_BB_JJ_AA_HH_FF_II': 2308, 'DD_EE_CC_BB_JJ_AA_HH_FF_II_GG': 684}
('DD_EE_CC_BB_JJ_AA_HH_FF_II_GG', 684)


In [1]:
### https://adventofcode.com/2022/day/16  ## 1754
# AoC 2022 - 16_1

### DRAFT #3

# ToDo: does not work right yet

import re
from itertools import permutations
        
def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]
           

def create_graph(_lines):
    _graph = dict()
    parse_out = ";|,|rate|="
    for line in _lines:
        line = line.split()
        node = re.sub(parse_out, "", line[1])
        rate = int(re.sub(parse_out, "", line[4]))
        neighbors = [re.sub(parse_out, "", l) for l in line[9:]]        
        _graph[node] = {'neighbors': neighbors, 'rate': rate}
    return _graph


def search_paths(_graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    paths = []
    for node in _graph[start]['neighbors']:
        if node not in path:
            newpaths = search_paths(_graph, node, end, path)
            for newpath in newpaths:
                    paths.append(newpath)
    return paths


def get_shortest_path_costs(_graph, start, end):
    return min([len(_path) for _path in search_paths(_graph, start, end)])
    
    
def get_all_node_pairs(_graph):
    combinations = []
    nodes = list(_graph.keys())
    for node in nodes:
        nodes = list(_graph.keys())
        nodes.pop(nodes.index(node))                  
        combinations += list(zip([node]*len(nodes), nodes))
    return list(set(combinations))


def calc_pressure(_graph, node_id, max_costs):
    costs = 0
    pressure = 0
    for node in node_id.split('_'):
        costs += 2 if _graph[node]['rate'] != 0 else 1
        pressure += (max_costs - costs) * _graph[node]['rate']
#         print("COSTS:", costs, pressure)
    return pressure


def get_sorted_node_degrees(_graph, _nodes):
    degrees = dict()
    for node in _nodes:
        degrees[node] = {'degree': len(graph[node]['neighbors']), 'rate': graph[node]['rate']}
    return dict(sorted(degrees.items(), key=lambda item: item[1]['degree'],reverse=True))    
    

def get_all_path_sorted_by_max_rate(_graph, _node_pairs):
    all_paths = dict()
    for start, end in node_pairs:
        paths = search_paths(_graph, start, end)
        for _path in paths:    
            sum_of_rates = 0
            opening_costs = 0
            for node in _path:
                rate = _graph[node]['rate']
                if rate != 0:
                    opening_costs += 1
                sum_of_rates += rate            
            all_paths['_'.join(_path)] = {'costs': len(_path) + opening_costs, 'sum_of_rates': sum_of_rates}
    return dict(sorted(all_paths.items(), key=lambda item: item[1]['sum_of_rates'],reverse=True))

    
def get_sorted_paths_n_tuples(sorted_paths, _num_nodes):
    n_tuples = [[] for col in range(_num_nodes + 1)] 
    for key, properties in sorted_paths.items():
        path_list = key.split('_')
        n_tuples[len(path_list)] += [(key, properties)]     
    return n_tuples


def filter_tuple_pairs(all_tuples, _node, _neighbors, tuple_size):
    return {k: v for k, v in all_tuples[tuple_size] for item in _neighbors if _node in k.split('_') and item in k.split('_')}


def filter_tuples(_graph, all_tuples, _selected_nodes, tuple_size, max_costs):
    filtered_tuples = dict()
#     print("tuple size:", tuple_size)
#     print("selected_nodes:", _selected_nodes)
#     print(f"all_tuples[{tuple_size}]:", all_tuples[tuple_size])
    path_candidates = dict()
    max_rate = 0
    for node_id, _tuple in all_tuples[tuple_size]:
        path_candidate = node_id.split('_')
        # all but maximum one nodes of selected tuple are in next tuple, + 1 new node of next tuple => 2 nodes can differ
        is_sublist = len(list(set(path_candidate).intersection(set(_selected_nodes)))) >= len(path_candidate) - 2
        if is_sublist:
            selected_nodes_neighbors = list(set([neighbor for node in _selected_nodes for neighbor in _graph[node]['neighbors']]))
#             print("NEIGHBORS:", selected_nodes_neighbors)
            if all(item in selected_nodes_neighbors for item in path_candidate):
#                 print(path_candidate)
    #             print(_tuple['costs'])
    #             print(_tuple['sum_of_rates'])
                if _tuple['costs'] <= max_costs:
                    max_rate = calc_pressure(_graph, node_id, max_costs)
                    path_candidates['_'.join(path_candidate)] = max_rate
#     print('PATH CANDIDATES:', path_candidates)
    path_candidates = dict(sorted(path_candidates.items(), key=lambda item: item[1],reverse=True))
#     print('PATH CANDIDATES:', path_candidates)
    best_candidate = list(path_candidates.keys())[0].split('_') if path_candidates else {}
    return best_candidate 
                            

def look_4_next_tuples(_graph, all_tuples, selected_path, path_before, tuple_size, _num_nodes, max_costs): 
#     for _path in selected_path:
    if selected_path == {}:
        return path_before
    if tuple_size > _num_nodes:
        return path_before
    next_tuple = filter_tuples(_graph, all_tuples, selected_path, tuple_size, max_costs)
#     print('NEXT TUPLE:', next_tuple)
    return look_4_next_tuples(_graph, all_tuples, next_tuple, selected_path, tuple_size + 1, _num_nodes, max_costs)

    

input_data = "aoc2022_data/aoc2022_16.txt"
# input_data = "aoc2022_data/example.txt"
lines = get_input_data(input_data)
time = 30

graph = create_graph(lines)
nodes = list(graph.keys())
num_nodes = len(nodes)

# greedy nodes degrees list sorted by maximum degrees
nodes_degrees = get_sorted_node_degrees(graph, nodes)

# get all possible pairs of nodes in graph
node_pairs = get_all_node_pairs(graph)
# get all possible paths in graph and sort them by maximum sum of rates
sorted_max_sum_paths = get_all_path_sorted_by_max_rate(graph, node_pairs)
# greedy list of paths list for each possible path length, each path list is sorted by maximal sum of rates
paths_n_tuples = get_sorted_paths_n_tuples(sorted_max_sum_paths, num_nodes)


# algorithm:
#     * get nodes with most neighbors (higest degrees)
#     * compare their sum of rates -> choose the best one(s)
#     * get neighbors of the node
#     * look at next n_tuples (here for 2-tuples) which contain this node and one of it's neighbors
#     * select the tuple(s) with the best sum of rates
#     * look at next n_tuples (here for 3-tuples) which contain this node and all nodes of the current path - 1 node for backtracking
#     * select the tuple(s) with the best sum of rates
#     * look at next n_tuples ...

max_pressures = dict()
for i in range(len(nodes)):
    max_degree = max([v['degree'] for v in nodes_degrees.values()])
    max_degree_nodes = {k: v for k, v in nodes_degrees.items() if v['degree'] == max_degree}
    max_degree_nodes_sorted = dict(sorted(max_degree_nodes.items(), key=lambda item: item[1]['rate'],reverse=True))
    if graph[nodes[i]]['rate'] > 0:
        for node in max_degree_nodes_sorted:
            neighbors = graph[node]['neighbors']
            count = 0
            n_tuple_size = 2
            filtered_n_tuples = filter_tuple_pairs(paths_n_tuples, node, neighbors, n_tuple_size)
            for filtered_tuple in filtered_n_tuples:
    #             if filtered_n_tuples[filtered_tuple]['sum_of_rates'] == 0:
                optimal_path_candidate = look_4_next_tuples(graph, paths_n_tuples, filtered_tuple.split('_'), node, (n_tuple_size + 1), num_nodes, time)
    #             print("optimal_path_candidate: ", optimal_path_candidate)
                optimal_path_candidate_id = '_'.join(optimal_path_candidate)
                max_pressures[optimal_path_candidate_id] = calc_pressure(graph, optimal_path_candidate_id, time)
print(f"Best candidates: {max_pressures}")
max_pressure = max(list(max_pressures.values()))
print(f"{max_pressure} is the most pressure which can be released.")      
        

Best candidates: {'VQ_NC_UI_ZG_SH_DU_NU_RN_WK_OV_QC_YS_CX_ZN_XI': 1502, 'OV_WK_RN_NU_HS_QC_VJ_CS_AA_MS_AH': 753, 'UI_NC_VQ_WK_RN_NU_DU_SH_ZG_XO_EA_QE_CX_ZN_XI': 1463, 'VQ_NC_UI_ZG_SH_DU_NU_RN_WK_XI_ZN_CX_OE_WC_AA_MS_AH': 1382, 'MW_PY_UB_EI_HD_JI_RA_TK_AA_AL_NU_AM_CX_OE_WC': 1054, 'CS_VJ_QC_HS_NU_DU_SH_ZG_XO_EA_QE_CX_ZN_XI_WK_AH_MS_AA_WC_OE': 900, 'VQ_NC_UI_ZG_SH_DU_NU_RN_WK_XI_ZN_CX_AM': 1382, 'CD_YH_VE_NM_DC_UE_XK_JC_XS_CE_WD_RA_TK_AA_AL_NU_HS_QC_VJ_CS': 1834, 'MS_AH_WK_XI_ZN_CX_QE_EA_GX_RA_TK_AA_WC_OE': 537, 'MW_PY_UB_EI_HD_JI_RA_TK_AA_CS_VJ_QC_YS_CX_OE_WC': 1027, 'CD_YH_VE_NM_DC_UE_XK_JC_XS_CE_WD_RA_TK_AA_CS_VJ_QC_YS_CX_OE_WC': 1784, 'CD_YH_VE_NM_DC_UE_XK_JC_XS_CE_WD_RA_TK_AA_AL_NU_RN_WK_AH_MS': 1849, 'VQ_NC_UI_ZG_SH_DU_NU_HS_QC_VJ_CS_AA_MS_AH_WK_XI_ZN_CX_OE_WC': 1430, 'MW_PY_UB_EI_HD_JI_RA_GX_EA_QE_CX_AM_NU_HS_QC_VJ_CS_AA_WC_OE': 1177}
1849 is the most pressure which can be released.


In [ ]:
### https://adventofcode.com/2022/day/17
# AoC 2022 - 17_1

# TODO later

In [ ]:
# AoC 2022 - 17_2

# TODO later

In [ ]:
### https://adventofcode.com/2022/day/18
# AoC 2022 - 18_1

# TODO later

In [ ]:
# AoC 2022 - 18_2

# TODO later

In [ ]:
### https://adventofcode.com/2022/day/19
# AoC 2022 - 19_1

# TODO later

In [ ]:
# AoC 2022 - 19_2

# TODO later

In [62]:
### https://adventofcode.com/2022/day/20
# AoC 2022 - 20_1

# DRAFT #1, example works, but real input is currently too low

def get_input_data(in_file: str):
    with open(in_file, 'r') as f_in:
        return [_line.strip() for _line in f_in.readlines()]


def mix_items(_items: list, alphabet_len: int, num_mixes: int):
    mixed_list = dict()
    current_list = [0] * alphabet_len
    for idx, item in enumerate(_items):
        mixed_list[str(idx)] = {
            'item': item, 
            'visited': False, 
            'current_idx': idx, 
            'new_idx': idx
        }
    for current_key, item in mixed_list.items():
        mv_distance = int(item['item'])
        dist_from_current_index = mv_distance + item['current_idx'] 
        if dist_from_current_index <= 0:
            dist_from_current_index -= 1
        if dist_from_current_index >= 6:
            dist_from_current_index += 1
        if item != 0:
            idx_to_move = (dist_from_current_index) % len_alphabet
            item['new_idx'] = idx_to_move
            print(f"move {item['item']} from index {item['current_idx']} to index {item['current_idx'] + mv_distance} => {item['new_idx']}")

            for key, other_item in mixed_list.items():
                if key != current_key:
                    # update current index
                    if other_item['current_idx'] > item['current_idx'] and other_item['current_idx'] <= item['new_idx']:
                        other_item['current_idx'] -= 1
                    if other_item['current_idx'] < item['current_idx'] and other_item['current_idx'] >= item['new_idx']:
                        other_item['current_idx'] += 1                        
                        
            item['visited'] = True
        item['current_idx'] = item['new_idx']
        for _, item in mixed_list.items():
            current_list[item['current_idx']] = item['item']
        print(current_list)
    return current_list
    
    
input_data = "aoc2022_data/aoc2022_20.txt"
input_data = "aoc2022_data/example.txt"
items = get_input_data(input_data)
mixed_lines = items
len_alphabet = len(items)
num_mixes = 1
steps = [1000, 2000, 3000]

for run in range(num_mixes):
    mixed_items = mix_items(items, len_alphabet, num_mixes)
#     print(mixed_items)
max_step = max(steps)
first_zero_idx = mixed_items.index('0')
extended_list = mixed_items * (int(max_step / len_alphabet) + 1 + first_zero_idx)
sum_grove_coord = 0
for step in steps:    
    step_val = int(extended_list[step + first_zero_idx])
    print(f"value at {step}th step after first zero: {step_val}")
    sum_grove_coord += step_val

print(f"The sum of the three numbers that form the grove coordinates is {sum_grove_coord}.")
    

move 1 from index 0 to index 1 => 1
['2', '1', '-3', '3', '-2', '0', '4']
move 2 from index 0 to index 2 => 2
['1', '-3', '2', '3', '-2', '0', '4']
move -3 from index 1 to index -2 => 4
['1', '2', '3', '-2', '-3', '0', '4']
move 3 from index 2 to index 5 => 5
['1', '2', '-2', '-3', '0', '3', '4']
move -2 from index 2 to index 0 => 6
['1', '2', '-3', '0', '3', '4', '-2']
move 0 from index 3 to index 3 => 3
['1', '2', '-3', '0', '3', '4', '-2']
move 4 from index 5 to index 9 => 3
['1', '2', '-3', '4', '0', '3', '-2']
value at 1000th step after first zero: 4
value at 2000th step after first zero: -3
value at 3000th step after first zero: 2
The sum of the three numbers that form the grove coordinates is 3.
